## Libraries

In [1]:
#Import All Dependencies
# import cv2, os, bz2, json, csv, difflib, requests, socket, whois, urllib.request, urllib.parse, urllib.error, re, OpenSSL, ssl
import numpy as np
from datetime import datetime
from urllib.parse import urlparse
from urllib.request import Request, urlopen
# from selenium import webdriver
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
# from timeout import timeout
import requests
import numpy as np
import urllib
import cv2
import re
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.options import Options
from PIL import Image
from io import BytesIO
import time
import os
import os.path
from os import path
import io
from difflib import SequenceMatcher

In [2]:
import contextlib 
  
try: 
    from urllib.parse import urlencode           
except ImportError: 
    from urllib import urlencode 
try: 
    from urllib.request import urlopen 
except ImportError: 
    from urllib2 import urlopen 

import sys 

### TinyURL

In [3]:
#Taken from https://www.geeksforgeeks.org/python-url-shortener-using-tinyurl-api/
#Returns the url subtracting the domain name with www and com stuff
#So, http://tinyurl.com/y5bffkh2 ---becomes---> y5bffkh2
def getTinyURL(URL): 
    request_url = ('http://tinyurl.com/api-create.php?' + urlencode({'url':URL}))     
    with contextlib.closing(urlopen(request_url)) as response:                       
        return response.read().decode('utf-8 ')[19:]

In [4]:
#Returns Beautiful Soup object
def getHTML(URL):
    try:
        hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36'} #Make the user agent verified, that is Mozilla
#         req = Request(URL,headers=hdr)
        req = requests.get(URL, headers=hdr)
        page = req.text #Get URL HTML contents
        soup = BeautifulSoup(page, 'html.parser') #Convert to BeutifulSoup
        print("Built Soup")
        #prettyText = str(soup.prettify()) #Convert the HTML in its form
        return soup
    except Exception as e:
# #         if e.__class__.__name__ == "TimeoutError": raise TimeoutError("")
        return None

### Selenium

In [5]:
fireFoxBinaryPath = r'C:\Program Files (x86)\Mozilla Firefox\firefox.exe'
geckoDriverPath = "C:/Users/Yash/Documents/geckodriver.exe"

googleChomreDriverPath = r"C:\Users\Yash\Documents\chromedriver.exe"

windowSizeXAxis = 1024
windowSizeYAxis = 768

In [6]:
# Uses Firefox to launch Selenium Headlessly: One needs to have:
# firefox binary path(located in Program Files folder(Windows)) and 
# geckoDriver, which acts as an intermediate between selenium and firefox, and can download it from: https://github.com/mozilla/geckodriver/releases
def getSeleniumDriverForFirefox(fireFoxBinaryPath, geckoDriverPath, windowSizeXAxis, windowSizeYAxis):
    options = Options()
    options.set_headless(headless=True)
    options.binary = binary
    options.add_argument('--ignore-certificate-errors')
    
    cap = DesiredCapabilities().FIREFOX
    cap["marionette"] = True #optional
    
    driver = webdriver.Firefox(firefox_options=options, capabilities=cap, executable_path=geckodriver)
    driver.set_window_size(windowSizeXAxis, windowSizeYAxis) # set the window size that you need 
    return driver

def getSeleniumDriverForChrome(googleChomreDriverPath):
    options = webdriver.ChromeOptions()
    options.headless = False

    driver = webdriver.Chrome(options=options, executable_path=googleChomreDriverPath)
    return driver

# Saves the screenshot with the last 20 characters of the URL as the file name
# Firefox doesn't properly returns the whole page's screenshot, so use Chrome
def takeScreenshotWithFirefox(URL):
    driver2 = getSeleniumDriverForChrome(googleChomreDriverPath)
    driver2.get(URL)
    driver2.save_screenshot("screenshots/" + getTinyURL(URL)+".png")
    driver2.quit()
    
def takeScreenshotWithChrome(URL, driver):
    driver.get(URL)

    S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
    driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment
    driver.find_element_by_tag_name('body').screenshot("screenshots/" + getTinyURL(URL)+".png")

### File Handling

In [7]:
# Create target directory & all intermediate directories if don't exists
def createdDirectory(path):
    try:
        os.mkdir(path)    
        print("Directory " , path ,  " Created ")
    except FileExistsError:
        print("Directory " , path ,  " already exists")  

### DOM

In [ ]:
# Unit Testing for XPath


In [10]:
#https://selenium-python.readthedocs.io/locating-elements.html#locating-by-xpath
#XPath can either id or name relative
def getXPath(currentTag):
    returnXPath = ""
    while(currentTag.parent!=None):
        if "id" in (currentTag.attrs):
            print("true")
            returnXPath = currentTag.name + "[@id='" + currentTag.attrs['id'].strip() + "']/" + returnXPath #//form[@id='loginForm']
            break
        if "name" in (currentTag.attrs):
            print("true")
            returnXPath = currentTag.name + "[@name='" + currentTag.attrs['name'].strip() + "']/" + returnXPath #//form[@id='loginForm']
            break 
        returnXPath = currentTag.name + "/" + returnXPath
        currentTag = currentTag.parent
        print(currentTag.attrs)
        
        
    returnXPath = returnXPath.replace("[document]/html","/") #When it reaches the end of document parent, it adds the following it it's start, so we need to delete it
    if not returnXPath.startswith("//"): #the XPath should start with 2 forward slash
        returnXPath = "//" + returnXPath
    if returnXPath.endswith("/"): #the XPath should not end with forward slash
        returnXPath = returnXPath[:-1] 
    returnXPath = returnXPath.replace("meta/","").replace("table/tr", "table/tbody/tr") #Few changes to be made while performing XPath
    return (returnXPath) #//div[@id='tab-panel-0-w3']/div/span/h2

#### Price

In [11]:
def getTheTagElementForPrice(gShopPriceUpdated, soup):
    returnPriceElementTag = None
    try:
        dummyVar = soup(text=re.compile(gShopPriceUpdated))
    #     print(dummyVar)
        for elem in dummyVar:
#             print("elem.parent",elem.parent)
#             print("elem.parent.name",elem.parent.name)
            if returnPriceElementTag == None: #The first element is the return value unless we encounter a heading tag
                returnPriceElementTag = elem.parent
            if "h" in elem.parent.name: #Found the heading tag, so return this tag and break the loop
                returnPriceElementTag = elem.parent
                return returnPriceElementTag
    except Exception as e:
        print("Error in getTheTagElementForPrice(gShopPriceUpdated, soup)")
    return returnPriceElementTag

def findPriceElementTag(gShopPrice, soup): #gShopPrice = $379.00
    gShopPrice = gShopPrice.replace("now","").strip() #GShop soemtimes gives prices with now suffix like "$0.00 now"
    print(gShopPrice)
    
    if "$" in gShopPrice:
        gShopPriceUpdated = gShopPrice.replace("$", "\$") #gShopPriceUpdated = \$379.00; Required because $ is reserved keyword for regex
        print(gShopPriceUpdated)
        returnPriceElementTag = getTheTagElementForPrice(gShopPriceUpdated, soup)
        if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
            return returnPriceElementTag
    
    gShopPriceUpdated = gShopPrice.replace("$","") #gShopPriceUpdated = 379.00
    print(gShopPriceUpdated)
    returnPriceElementTag = getTheTagElementForPrice(gShopPriceUpdated, soup)
    if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
        return returnPriceElementTag
        
    gShopPriceUpdated = gShopPrice.replace("$", "\$").split(".")[0] #gShopPriceUpdated = \$379
    print(gShopPriceUpdated)
    returnPriceElementTag = getTheTagElementForPrice(gShopPriceUpdated, soup)
    if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
        return returnPriceElementTag
        
    gShopPriceUpdated = gShopPrice.replace("$","").split(".")[0] #gShopPriceUpdated = 379
    print(gShopPriceUpdated)
    returnPriceElementTag = getTheTagElementForPrice(gShopPriceUpdated, soup)
    if returnPriceElementTag != None and len(str(returnPriceElementTag))<400:
        return returnPriceElementTag
        
    return None

#### Heading

In [12]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [13]:
def findHeadingElementTag(gShopPrice, soup):
    maxSimilarRatio = 0
    resultHeadingElementTag = None

    allHeadingTags = soup.find_all(re.compile(r'h\d+'))
    for headingTag in allHeadingTags:
    #     print(headingTag)
        if headingTag.string!=None:
            if maxSimilarRatio<similar(gShopPrice, headingTag.string):
                maxSimilarRatio = similar(gShopPrice, headingTag.string)
                resultHeadingElementTag = headingTag
    #Threshold
    if maxSimilarRatio<0.2:
        allHeadingTags = soup.find_all(re.compile(r'span'))
        for headingTag in allHeadingTags:
        #     print(headingTag)
            if headingTag.string!=None:
                if maxSimilarRatio<similar(gShopPrice, headingTag.string):
                    maxSimilarRatio = similar(gShopPrice, headingTag.string)
                    resultHeadingElementTag = headingTag
                
    print("resultHeadingElementTag:",resultHeadingElementTag)
    print("maxSimilarRatio:",maxSimilarRatio)
    
    return resultHeadingElementTag

### Testing

In [14]:
GShopBasePath = r"C:\Users\Yash\Desktop\UIUC_RA\GoogleShoppingScraper\Verson 3\GShop"
listOfGShopProducts = os.listdir(GShopBasePath)
listOfGShopProducts

['y2vz9rq4',
 'y4qb2hdc',
 'y5o4x866',
 'y6edc2lh',
 'y6lfq3th',
 'y6x87t8r',
 'yxbqxgs4',
 'yxdnd79k',
 'yxtgxmtl',
 'yyblu48a',
 'yylycg4u']

In [15]:
for GShopProducts in listOfGShopProducts:
    datalogsPath = GShopBasePath + "\\" + GShopProducts + "\datalogs"
    listOfDatalogs = os.listdir(datalogsPath)
    screenshotsImagesPath = GShopBasePath + "\\" + GShopProducts + "\screenshots"
    listOfScreenshots = os.listdir(screenshotsImagesPath)
    
    for screenshotImagesNames in listOfScreenshots: #Make bounding box for each image
        
        #There can be cases where the screenshot was successful, but price could not be obtained
        if not path.exists(datalogsPath + "/" + screenshotImagesNames[:-4] + ".txt"):
            continue
        
        #Get the URL of current screenshot
        print("http://tinyurl.com/" + screenshotImagesNames[:-4])
        URLOfCurrentScreenshot = "http://tinyurl.com/" + screenshotImagesNames[:-4]
        
        #Get the Price of current screenshot
        file = open(datalogsPath + "/" + screenshotImagesNames[:-4] + ".txt", "r", encoding='utf-8') 
        priceOfCurrentScreenshot = str(file.read()).split(" ,;, ")[1]
        print("priceOfCurrentScreenshot:",priceOfCurrentScreenshot)
        
        #Get the Heading of current screenshot
        file = open(datalogsPath + "/" + screenshotImagesNames[:-4] + ".txt", "r", encoding='utf-8') 
        headingeOfCurrentScreenshot = str(file.read()).split(" ,;, ")[0].strip()
        print("headingeOfCurrentScreenshot:",headingeOfCurrentScreenshot)
        
        #Find Price Element
        soup = getHTML(URLOfCurrentScreenshot)
        returnPriceElementTag = findPriceElementTag(priceOfCurrentScreenshot, soup)
        print(returnPriceElementTag)
        print(len(str(returnPriceElementTag)))
        
        #Find Heading Element
        returnHeadingElementTag = findHeadingElementTag(headingeOfCurrentScreenshot, soup)
        print(returnHeadingElementTag)
        print(len(str(returnHeadingElementTag)))        
        
#         try:
        
        #If priceElement is not None, then find it's location and take it's cropped screenshot and save
        if returnPriceElementTag!=None:

            #Get XPath
            # For Price
            XPathForPriceElementTag = getXPath(returnPriceElementTag)
            print("XPathForPriceElementTag:",XPathForPriceElementTag) 
            # For Price
            XPathForHeadingElementTag = getXPath(returnHeadingElementTag)
            print("XPathForHeadingElementTag:",XPathForHeadingElementTag)


            #Get the first line of dataLog: Helps in rewriting the XPath
            f = open(datalogsPath + "/" + screenshotImagesNames[:-4] + ".txt","r", encoding='utf-8')
            first_line = f.readline()
            first_line = first_line.split("PriceXPath")[0]
            first_line = first_line.split("\n")[0]
            print("first_line:",first_line)
            f.close()
            
            #Reopen the dataLog freshly and rewrite the whole thing
            f = open(datalogsPath + "/" + screenshotImagesNames[:-4] + ".txt","w", encoding='utf-8')
            
            #Write the Heading ,;, Price
            f.write(first_line)
    
            #Write with "PriceXPath: <XPath>"
            f.write("\nPriceXPath: " + str(XPathForPriceElementTag))

            #Write with "HeadingXPath: <XPath>"
            f.write("\nHeadingXPath: " + str(XPathForHeadingElementTag))

            print("WROTEEEEEEEEEEEEEEEE")
            f.close()

#         except Exception as e:
#             print("Driver Failed")
#             print(e)
    

http://tinyurl.com/y54a4wl9
priceOfCurrentScreenshot: $15.00/mo
headingeOfCurrentScreenshot: Samsung Galaxy S10 Plus - 512 GB - Ceramic White - Sprint - CDMA/GSM
Built Soup
$15.00/mo
\$15.00/mo
15.00/mo
\$15
15
None
4
resultHeadingElementTag: <h1 class="mt-0 mb-10 text-center" device-name="">Samsung Galaxy S10+</h1>
maxSimilarRatio: 0.41379310344827586
<h1 class="mt-0 mb-10 text-center" device-name="">Samsung Galaxy S10+</h1>
74
http://tinyurl.com/y6r8bxfz
priceOfCurrentScreenshot: $1,249.99
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
headingeOfCurrentScreenshot: Samsung Galaxy S10 Plus - 512 GB - Ceramic White - Sprint - CDMA/GSM
Built Soup
$1,249.99
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
\$1,249.99
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
1,249.99
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div

Built Soup
$139.00
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/h1
\$139.00
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/h1
139.00
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/h1
\$139
<h2 class="display-price">$139.00</h2>
38
resultHeadingElementTag: <h1 class="item-title">Samsung Galaxy J7 Refine 32GB GOLD BoostMobile (Open Box)</h1>
maxSimilarRatio: 0.7663551401869159
<h1 class="item-title">Samsung Galaxy J7 Refine 32GB GOLD BoostMobile (Open Box)</h1>
85
{'class': ['item-price']}
{'class': ['item-price-logistics-wrapper']}
{'class': ['item-price-wrapper'], 'id': 'tab-panel-0-w2', 'data-widget': '/prpweb$0.0.16/src/modules/vi-components/app-theme-item-price/index'}
true
XPathForPriceElementTag: //div[@id='tab-panel

Built Soup
$645.39
PriceXPath: /html/body/div/input/div[2]/div/div/div/div[2]/div/div[4]/div[3]/div/div[2]/div/div/span/span
HeadingXPath: /html/body/div/input/div[2]/div/div/div/div/div[2]/div/div/div/h1
\$645.39
PriceXPath: /html/body/div/input/div[2]/div/div/div/div[2]/div/div[4]/div[3]/div/div[2]/div/div/span/span
HeadingXPath: /html/body/div/input/div[2]/div/div/div/div/div[2]/div/div/div/h1
645.39
PriceXPath: /html/body/div/input/div[2]/div/div/div/div[2]/div/div[4]/div[3]/div/div[2]/div/div/span/span
HeadingXPath: /html/body/div/input/div[2]/div/div/div/div/div[2]/div/div/div/h1
\$645
<span class="priceDisplay">$645.39  </span>
43
resultHeadingElementTag: <h1 class="pagetitle">Samsung Notebook 5 AMD Ryzen 5 2500U 2.0GHz 8GB 1TB ac BT WC 15.6" FHD W10H</h1>
maxSimilarRatio: 0.5862068965517241
<h1 class="pagetitle">Samsung Notebook 5 AMD Ryzen 5 2500U 2.0GHz 8GB 1TB ac BT WC 15.6" FHD W10H</h1>
102
{'class': ['product-price']}
{'style': 'padding:0;', 'class': ['col-xs-6', 'col-sm-

Built Soup
$466.97
\$466.97
466.97
\$466
466
None
4
resultHeadingElementTag: None
maxSimilarRatio: 0
None
4
http://tinyurl.com/y2tsagfh
priceOfCurrentScreenshot: $737.99
PriceXPath: /html/head/title
HeadingXPath: /html/body/div[3]/table/tr/td[2]/div[2]/form/h1
headingeOfCurrentScreenshot: Samsung Galaxy S10 - 128 GB - Prism Black - Sprint - CDMA/GSM
Built Soup
$737.99
PriceXPath: /html/head/title
HeadingXPath: /html/body/div[3]/table/tr/td[2]/div[2]/form/h1
\$737.99
PriceXPath: /html/head/title
HeadingXPath: /html/body/div[3]/table/tr/td[2]/div[2]/form/h1
737.99
PriceXPath: /html/head/title
HeadingXPath: /html/body/div[3]/table/tr/td[2]/div[2]/form/h1
\$737
<title>Samsung Galaxy S10 - 128 GB - Prism Black - Sprint [SM-G973U] - $737.99 : Unlocked Cell Phones, GSM, CDMA, No-Contracts! | Cell2Get</title>
146
resultHeadingElementTag: <h1 class="productGeneral" id="productName">Samsung Galaxy S10 - 128 GB - Prism Black - Sprint</h1>
maxSimilarRatio: 0.9009009009009009
<h1 class="productGene

Built Soup
$439.00
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[4]/div/div[2]/section/div/div[2]/div/ul/li[3]/div/div[2]/h3
\$439.00
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[4]/div/div[2]/section/div/div[2]/div/ul/li[3]/div/div[2]/h3
439.00
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[4]/div/div[2]/section/div/div[2]/div/ul/li[3]/div/div[2]/h3
\$439
<h2 class="display-price">$439.00</h2>
38
resultHeadingElementTag: <h3><a data-navsrc-tracking='[{"eventFamily":"PRP","eventAction":"ACTN","actionKind":"NAVSRC","operationId":"2385738","flushImmediately":false,"eventProperty":{"moduledtl":"mi:4475|li:8293","parentrq":"2914c1af16e0a4e921e50030fff86966","pageci":"9ce9484d-f718-48dd-ac76-38ce08a476d2"}},{"actionKind":"NAVSRC"}]' href="https://www.ebay.com/p/23536

...

Built Soup
$197.99
PriceXPath: /html/body/div/div/div[2]/div[2]/div/div[3]/div[2]/form/div[2]/div[2]/div[2]/div/div[2]/div[2]/div/p/span/span
HeadingXPath: /html/body/div/div/div[2]/div[2]/div/div[3]/div[2]/form/div[2]/div[2]/div/div/h1
\$197.99
PriceXPath: /html/body/div/div/div[2]/div[2]/div/div[3]/div[2]/form/div[2]/div[2]/div[2]/div/div[2]/div[2]/div/p/span/span
HeadingXPath: /html/body/div/div/div[2]/div[2]/div/div[3]/div[2]/form/div[2]/div[2]/div/div/h1
197.99
PriceXPath: /html/body/div/div/div[2]/div[2]/div/div[3]/div[2]/form/div[2]/div[2]/div[2]/div/div[2]/div[2]/div/p/span/span
HeadingXPath: /html/body/div/div/div[2]/div[2]/div/div[3]/div[2]/form/div[2]/div[2]/div/div/h1
\$197
<span class="price" id="product-final_price">$197.99</span>
59
resultHeadingElementTag: <h1 id="product-title" itemprop="name">Samsung Gear Fit2 Pro Smart Fitness Band (Large, Red)</h1>
maxSimilarRatio: 0.53125
<h1 id="product-title" itemprop="name">Samsung Gear Fit2 Pro Smart Fitness Band (Large, Red)</

Built Soup
$856.79
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
\$856.79
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
856.79
PriceXPath: /div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
\$856
<h2 class="display-price">$856.79</h2>
38
resultHeadingElementTag: <h1 class="product-title">Samsung Notebook 9 Pro NP940X5N 15" (256 GB, Intel Core i7 8th Gen., 4 GHz, 16 GB) - Titan Silver - NP940X5N-X01US</h1>
maxSimilarRatio: 0.6266094420600858
<h1 class="product-title">Samsung Notebook 9 Pro NP940X5N 15" (256 GB, Intel Core i7 8th Gen., 4 GHz, 16 GB) - Titan Silver - NP940X5N-X01US</h1>
145
{'class': ['item-price']}
{'class': ['item-price-logistics-wrapper']}
{'class': ['item-price-wrapper'], 'id': 'tab-panel-1-w3', 'data-widget': '/prpweb$0.0.16/src/modules/vi-components/app-theme-item-price/index'}
true
XPathForPriceElementTag: //div[@id='tab-panel-1-w3']/div/sp

Built Soup
$579.00
PriceXPath: /html/body/div/main/section/div[2]/div[2]/div/div/p[2]
HeadingXPath: /html/body/div/main/section/div[2]/div[2]/div/div/h1
\$579.00
PriceXPath: /html/body/div/main/section/div[2]/div[2]/div/div/p[2]
HeadingXPath: /html/body/div/main/section/div[2]/div[2]/div/div/h1
579.00
PriceXPath: /html/body/div/main/section/div[2]/div[2]/div/div/p[2]
HeadingXPath: /html/body/div/main/section/div[2]/div[2]/div/div/h1
\$579
<p class="product__info-price">$579.00 USD</p>
46
resultHeadingElementTag: <h1 class="product__info-title">Samsung XE510C24-K01US Chromebook Pro</h1>
maxSimilarRatio: 0.4028776978417266
<h1 class="product__info-title">Samsung XE510C24-K01US Chromebook Pro</h1>
74
{'class': ['product__info']}
{'class': ['product__sticky-wrapper', 'product__sticky']}
{'class': ['grid__item', 'grid__item-xs-12-12', 'grid__item-md-5-12', 'product__col']}
{'class': ['grid']}
{'class': ['product']}
{}
{'id': 'app'}
true
XPathForPriceElementTag: //div[@id='app']/main/section

Built Soup
$628.49
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
\$628.49
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
628.49
PriceXPath: /html/body/div[2]/div[3]/div[3]/div[2]/div/div/div[2]/div/div/div/div/span/h2
HeadingXPath: /html/body/div[2]/div[3]/div[3]/div/div/div/h1
\$628
<h2 class="display-price">$628.49</h2>
38
resultHeadingElementTag: <h1 class="product-title">Samsung Chromebook Pro 12.3 inch 32GB 4GB 2.2GHz Notebook - Black</h1>
maxSimilarRatio: 0.4431137724550898
<h1 class="product-title">Samsung Chromebook Pro 12.3 inch 32GB 4GB 2.2GHz Notebook - Black</h1>
96
{'class': ['item-price']}
{'class': ['item-price-logistics-wrapper']}
{'class': ['item-price-wrapper'], 'id': 'tab-panel-0-w2', 'data-widget': '/prpweb$0.0.16/src/modules/vi-components/app-theme-item-price/

Built Soup
$1,451.99
\$1,451.99
1,451.99
\$1,451
1,451
None
4
resultHeadingElementTag: <span id="grpDescrip_0GA-001P-00076" style="display: inline;">
                samsung notebook 9 pro 13.3" touchscreen  i77500u  8gb  128gb ssd  silver  spen
            </span>
maxSimilarRatio: 0.44155844155844154
<span id="grpDescrip_0GA-001P-00076" style="display: inline;">
                samsung notebook 9 pro 13.3" touchscreen  i77500u  8gb  128gb ssd  silver  spen
            </span>
180
http://tinyurl.com/y6nwxmnb
priceOfCurrentScreenshot: $1,263.99
headingeOfCurrentScreenshot: Samsung Notebook 9 Pro 940X3MI 13.3″ Convertible Notebook - Core i7 7500U 2.7 GHz - 8 GB RAM - 128 GB SSD - Titan Silver
Built Soup
$1,263.99
\$1,263.99
1,263.99
\$1,263
1,263
None
4
resultHeadingElementTag: <span>Samsung Notebook 9 Pro 13.3" Touch-Screen - i7-7500U - 8GB - 128GB SSD - Silver - Spen</span>
maxSimilarRatio: 0.6504854368932039
<span>Samsung Notebook 9 Pro 13.3" Touch-Screen - i7-7500U - 8GB - 128GB SSD 